In [2]:
%pip install sentence_transformers umap umap-learn langchain cohere faiss textract moviepy google-cloud-speech pandas

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install opencv-python-headless ipywidgets pytube

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install --upgrade moviepy
!pip install librosa

import whisper
import time
import librosa
import soundfile as sf
import re
import os
import pandas as pd

base_model = whisper.load_model("base.en")
small_model = whisper.load_model("small.en")

  Cloning https://github.com/openai/whisper.git to /private/var/folders/m8/2wxq66fs2sbghj_z2wq8t2_40000gn/T/pip-req-build-4sjxcnm9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/m8/2wxq66fs2sbghj_z2wq8t2_40000gn/T/pip-req-build-4sjxcnm9
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801345 sha256=d8dddd6af33b261619626a77b7bd3742f606ca854b765a94bb2ba1f3fbd16004
  Stored in directory: /private/var/folders/m8/2wxq66fs2sbghj_z2wq8t2_40000gn/T/pip-ephem-wheel-cache-w3ljhbxn/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg

In [37]:
import os
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip

input_folder = "files/"
output_folder = "fragments/"
os.makedirs(output_folder, exist_ok=True)

def split_video_into_fragments(input_video_file, fragment_duration):
    fragment_num = 61

    clip = VideoFileClip(input_video_file)
    duration = int(clip.duration)

    fragment_start = 0
    fragment_end = fragment_duration

    while fragment_end <= duration:
        output_file = os.path.join(output_folder, f"fragment_{fragment_num}.mov")
        ffmpeg_extract_subclip(input_video_file, fragment_start, fragment_end, targetname=output_file)

        print(f"Processed fragment {fragment_num}")

        fragment_start += fragment_duration
        fragment_end += fragment_duration
        fragment_num += 1

    clip.close()

for filename in os.listdir(input_folder):
    if filename.endswith(".MOV"):
        input_video_file = os.path.join(input_folder, filename)
        split_video_into_fragments(input_video_file, 10)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Processed fragment 61
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Processed fragment 62
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Processed fragment 63
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Processed fragment 64
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Processed fragment 65


In [38]:
transcriptions = []
file_names = []

for i in range(61, 66):
    video_file = f'fragment_{i}.mov'
    video_path = "fragments/" + video_file
    audio_path = "audio/" + video_file[:-4] + ".wav"

    y, sr = librosa.load(video_path, sr=16000)
    sf.write(audio_path, y, sr)

    result = small_model.transcribe(audio_path)
    text = result["text"].strip()

    text_file = video_file[:-4] + ".txt"
    text_path = "video_fragments/" + text_file
    transcriptions.append(text)
    file_names.append(video_file)

In [32]:
%pip install sentence-transformers
%pip install umap-learn opencv-python Pillow
import umap.umap_ as umap

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [39]:
from transformers import pipeline
import cv2
from PIL import Image
import os

captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
interval = 10
output_directory = "frame_images"
os.makedirs(output_directory, exist_ok=True)

clip_results = []

for i in range(61, 66):
    video_file = f'fragment_{i}.mov'
    video_path = "fragments/" + video_file
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_skip = fps * interval
    frame_count = 0
    texts = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frames_to_skip == 0:
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            captions = captioner(image)
            if captions:
                texts.append(captions[0]['generated_text'])
        frame_count += 1

    cap.release()
    long_sentence = ' '.join(texts)

    clip_results.append(long_sentence)

In [40]:
clip_results

['a blue table tennis table a ping po table tennis game',
 'a ping pong table tennis game a blue table tennis court',
 'a blue table tennis court a ping po table tennis ball on a blue table',
 'a ping pong table tennis ball on a blue table a tennis player is hitting the ball',
 'a tennis player is about to hit the ball a skateboarder is riding down a ramp']

In [41]:
df = pd.DataFrame({'File Name': file_names, 'Transcription': transcriptions, 'Context': clip_results })
df.to_csv('df2.csv')
df

,File Name,Transcription,Context
0,fragment_61.mov,That's crazy,a blue table tennis table a ping po table tenn...
1,fragment_62.mov,p,a ping pong table tennis game a blue table ten...
2,fragment_63.mov,That's crazy. That's crazy. So there's this co...,a blue table tennis court a ping po table tenn...
3,fragment_64.mov,in that video is crazy,a ping pong table tennis ball on a blue table ...
4,fragment_65.mov,No. What if I say you got a cigar? You should ...,a tennis player is about to hit the ball a ska...
